Assess the stop sign location

remove the edge measurements:
"top_row" >6 AND "bottom_row" <762 AND "left_col" >6 AND "right_col" <1018

In [3]:
import geopandas as gpd

point_fname = r"F:\OneDrive_Emory\OneDrive - Emory\Research_doc\street_image_mapping\Maryland\Results\measured_stop_sign_076_joined.shp"

stop_sign_gdf = gpd.read_file(point_fname)
stop_sign_gdf

,XCoord,YCoord,CLUSTER_ID,NEAR_FID,NEAR_DIST,NEAR_X,NEAR_Y,NEAR_ANGLE,geometry
0,426043.08085,199095.134700,1,16211,1.153423,426043.737320,199094.186281,-55.312120,POINT (426043.081 199095.135)
1,425923.35185,157848.984200,2,134,1.328342,425924.661671,157849.205545,9.592076,POINT (425923.352 157848.984)
2,430187.91090,172716.153800,3,9163,1.965869,430188.847356,172714.425306,-61.552239,POINT (430187.911 172716.154)
3,404460.73730,184339.488067,4,5573,1.724357,404460.261278,184337.830749,-106.025037,POINT (404460.737 184339.488)
4,420535.54915,130913.158300,5,14046,0.879076,420535.480267,130912.281931,-94.497469,POINT (420535.549 130913.158)
...,...,...,...,...,...,...,...,...,...
10582,386722.05500,189882.862400,9020646,6912,0.505137,386721.840202,189882.405207,-115.165027,POINT (386722.055 189882.862)
10583,409617.05950,221979.707400,9020648,11041,1.600972,409616.386369,221978.254814,-114.863090,POINT (409617.060 221979.707)
10584,491939.42580,97516.605300,9020650,6291,6.233135,491933.283587,97515.544546,-170.201728,POINT (491939.426 97516.605)
10585,429899.21360,195295.500000,9020652,14699,2.101059,429900.993912,195296.615768,32.076449,POINT (429899.214 195295.500)


In [5]:
# Paired t-test: Measured (XCoord, YCoord) vs Ground Truth (NEAR_X, NEAR_Y)
from scipy import stats
import numpy as np

print("="*60)
print("PAIRED T-TEST: Measured vs Ground Truth Coordinates")
print("="*60)
print(f"Sample size: n = {len(stop_sign_gdf)}")

# 1. X-Coordinate comparison
x_ttest = stats.ttest_rel(stop_sign_gdf['XCoord'], stop_sign_gdf['NEAR_X'])
x_diff = stop_sign_gdf['XCoord'] - stop_sign_gdf['NEAR_X']
print(f"\n1. X-COORDINATE COMPARISON")
print(f"   Measured X mean:     {stop_sign_gdf['XCoord'].mean():.4f} m")
print(f"   Ground Truth X mean: {stop_sign_gdf['NEAR_X'].mean():.4f} m")
print(f"   Mean difference:     {x_diff.mean():.4f} m")
print(f"   Std of difference:   {x_diff.std():.4f} m")
print(f"   t-statistic: {x_ttest.statistic:.4f}")
print(f"   p-value: {x_ttest.pvalue:.6f}")
print(f"   Result: {'SIGNIFICANT' if x_ttest.pvalue < 0.05 else 'NOT significant'} (α = 0.05)")

# 2. Y-Coordinate comparison
y_ttest = stats.ttest_rel(stop_sign_gdf['YCoord'], stop_sign_gdf['NEAR_Y'])
y_diff = stop_sign_gdf['YCoord'] - stop_sign_gdf['NEAR_Y']
print(f"\n2. Y-COORDINATE COMPARISON")
print(f"   Measured Y mean:     {stop_sign_gdf['YCoord'].mean():.4f} m")
print(f"   Ground Truth Y mean: {stop_sign_gdf['NEAR_Y'].mean():.4f} m")
print(f"   Mean difference:     {y_diff.mean():.4f} m")
print(f"   Std of difference:   {y_diff.std():.4f} m")
print(f"   t-statistic: {y_ttest.statistic:.4f}")
print(f"   p-value: {y_ttest.pvalue:.6f}")
print(f"   Result: {'SIGNIFICANT' if y_ttest.pvalue < 0.05 else 'NOT significant'} (α = 0.05)")

# 3. Euclidean distance error
euclidean_error = np.sqrt(x_diff**2 + y_diff**2)
print(f"\n3. EUCLIDEAN DISTANCE ERROR")
print(f"   Mean error:   {euclidean_error.mean():.4f} m")
print(f"   Median error: {euclidean_error.median():.4f} m")
print(f"   Std error:    {euclidean_error.std():.4f} m")
print(f"   Max error:    {euclidean_error.max():.4f} m")

# One-sample t-test: Is mean location error significantly different from 0?
loc_ttest = stats.ttest_1samp(euclidean_error, 0)
print(f"   One-sample t-test (H0: mean error = 0):")
print(f"   t-statistic: {loc_ttest.statistic:.4f}, p-value: {loc_ttest.pvalue:.2e}")

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"X-coord: p = {x_ttest.pvalue:.6f} -> {'Significant' if x_ttest.pvalue < 0.05 else 'Not significant'}")
print(f"Y-coord: p = {y_ttest.pvalue:.6f} -> {'Significant' if y_ttest.pvalue < 0.05 else 'Not significant'}")

PAIRED T-TEST: Measured vs Ground Truth Coordinates
Sample size: n = 10587

1. X-COORDINATE COMPARISON
   Measured X mean:     428515.2077 m
   Ground Truth X mean: 428515.6020 m
   Mean difference:     -0.3943 m
   Std of difference:   1.5091 m
   t-statistic: -26.8857
   p-value: 0.000000
   Result: SIGNIFICANT (α = 0.05)

2. Y-COORDINATE COMPARISON
   Measured Y mean:     155836.5896 m
   Ground Truth Y mean: 155835.3943 m
   Mean difference:     1.1953 m
   Std of difference:   1.4994 m
   t-statistic: 82.0264
   p-value: 0.000000
   Result: SIGNIFICANT (α = 0.05)

3. EUCLIDEAN DISTANCE ERROR
   Mean error:   1.9725 m
   Median error: 1.5064 m
   Std error:    1.4896 m
   Max error:    9.9937 m
   One-sample t-test (H0: mean error = 0):
   t-statistic: 136.2495, p-value: 0.00e+00

SUMMARY
X-coord: p = 0.000000 -> Significant
Y-coord: p = 0.000000 -> Significant


In [7]:
width_fname = r"F:\OneDrive_Emory\OneDrive - Emory\Research_doc\street_image_mapping\DC_roads\manual_width.shp"

width_gdf = gpd.read_file(width_fname)
width_gdf

,Id,length1,length_dif,NEAR_FID,NEAR_DIST,abs_dif,raw_len,dif_square,dif_ratio,ratio_sq,geometry
0,0,10.351916,-0.651916,0,0.392815,0.651916,9.70,0.424994,0.067208,0.004517,"LINESTRING (406947.828 135392.209, 406957.911 ..."
1,0,7.552589,-1.052589,1,0.310265,1.052589,6.50,1.107943,0.161937,0.026224,"LINESTRING (401207.067 138037.264, 401212.660 ..."
2,0,9.397683,0.152317,2,0.477368,0.152317,9.55,0.023200,0.015949,0.000254,"LINESTRING (394362.656 144946.000, 394353.258 ..."
3,0,12.622456,-6.122456,3,0.295530,6.122456,6.50,37.484466,0.941916,0.887206,"LINESTRING (404806.591 136953.909, 404815.486 ..."
4,0,9.260729,-0.110729,4,0.500085,0.110729,9.15,0.012261,0.012102,0.000146,"LINESTRING (398997.763 143484.319, 398988.503 ..."
...,...,...,...,...,...,...,...,...,...,...,...
288,0,9.168104,-0.318104,302,0.495361,0.318104,9.95,0.101190,0.031970,0.001022,"LINESTRING (397876.102 145515.746, 397885.270 ..."
289,0,10.590832,0.109168,300,0.876174,0.109168,8.85,0.011918,0.012335,0.000152,"LINESTRING (400112.759 143391.677, 400120.760 ..."
290,0,8.996652,-1.546652,298,0.639648,1.546652,10.70,2.392131,0.144547,0.020894,"LINESTRING (395349.781 144706.062, 395358.714 ..."
291,0,9.142384,-0.892384,292,2.006572,0.892384,7.45,0.796349,0.119783,0.014348,"LINESTRING (398020.499 145325.230, 398020.457 ..."


In [ ]:
# Paired t-test: Measured (length1) vs Ground Truth (length1 + length_dif)
from scipy import stats

# Calculate ground truth length
measured = width_gdf['length1']
ground_truth = width_gdf['length1'] + width_gdf['length_dif']

print("="*60)
print("PAIRED T-TEST: Measured vs Ground Truth Length")
print("="*60)
print(f"Sample size: n = {len(width_gdf)}")

print(f"\nMeasured length mean:     {measured.mean():.4f} m")
print(f"Ground truth length mean: {ground_truth.mean():.4f} m")
print(f"Mean difference:          {width_gdf['length_dif'].mean():.4f} m")
print(f"Std of difference:        {width_gdf['length_dif'].std():.4f} m")

# Paired t-test
ttest_result = stats.ttest_rel(measured, ground_truth)
print(f"\nt-statistic: {ttest_result.statistic:.4f}")
print(f"p-value: {ttest_result.pvalue:.6f}")

if ttest_result.pvalue < 0.05:
    print(f"Result: SIGNIFICANT difference (p < 0.05)")
else:
    print(f"Result: NO significant difference (p >= 0.05)")